# Generate Data for Chartbook

Brian Dew

@bd_econ

In [1]:
import sys
sys.path.append('../src')

import time

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Equities Index Data from Yahoo! Finance

In [265]:
ltdate = int(time.time())
indices = ['GSPC', 'IXIC', 'DJI', 'RUT']
raw = pd.DataFrame()
for s in indices:
    url = ('https://query1.finance.yahoo.com/v7/finance/download/'+
           f'%5E{s}?period1=599616000&period2={ltdate}&'+
           'interval=1d&events=history')
    df = pd.read_csv(url, index_col='Date', parse_dates=True)
    df = df[~df.index.duplicated(keep='last')]
    raw[f'{s}_close'] = df['Adj Close']
    raw[f'{s}_volume'] = df['Volume']
raw.to_csv(data_dir / 'equity_indices_raw.csv', index_label='date')

In [266]:
closecol = [f'{s}_close' for s in indices]
raw = pd.read_csv(data_dir / 'equity_indices_raw.csv', 
                  index_col='date', parse_dates=True)[closecol] / 1_000

rename = {f'{s}_close': f'{s}_MA' for s in indices}
(raw.join(raw.loc['2017':]
          .rolling(252).mean()
          .rename(rename, axis=1))
    .loc['2018':]
    .to_csv(data_dir / 'equity_indices_lt.csv', 
            index_label='date', float_format='%g'))
data = raw.resample('MS').mean().append(raw.iloc[-1])
res = data.rename({f'{s}_close': s for s in indices}, axis=1)
res.to_csv(data_dir / 'equity_indices.csv', index_label='date', 
            float_format='%g')
rename = {f'{s}_close': f'{s}_MA' for s in indices}
(raw.join(raw.loc['2017':]
          .rolling(252).mean()
          .rename(rename, axis=1))
    .loc['2018':]
    .to_csv(data_dir / 'equity_indices_lt.csv', 
            index_label='date', float_format='%g'))
ch = pd.DataFrame()
for year in range(2017, 2023):
    yr = str(year)
    py = str(year - 1)
    s = (((raw.loc[yr].iloc[-1] / 
               raw.loc[py].iloc[-1]) - 1) * 100).rename(yr).apply('{:.1f}'.format)
    ch = ch.append(s)
ch = ch.append((raw.iloc[-252:].mean() * 1000).rename('1-year moving average').apply('{:,.0f}'.format))#
ch = ch.append((raw.iloc[-1] * 1000).rename(dtxt(raw.index[-1])['day2']).apply('{:,.0f}'.format))#
names = {'GSPC_close': '\hspace{0.1mm} {\color{green!80!blue!90!black}\\textbf{---}} \ S\&P 500', 
         'IXIC_close': '\hspace{0.1mm} {\color{blue}\\textbf{---}} \ Nasdaq', 
         'DJI_close': '\hspace{0.1mm} {\color{red}\\textbf{---}} \ Dow 30', 
         'RUT_close': '\hspace{0.1mm} {\color{violet}\\textbf{---}} \ Russell 3000', }
ch = (ch.rename({'2022': '2022 YTD'}).iloc[::-1].T)
ch = (ch.reindex(list(names.keys())).rename(names))
ch.to_csv(data_dir / 'equities.tex', sep='&', line_terminator='\\\ ', quotechar=' ')

/tmp/ipykernel_90747/3309950994.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = raw.resample('MS').mean().append(raw.iloc[-1])
/tmp/ipykernel_90747/3309950994.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ch = ch.append(s)
/tmp/ipykernel_90747/3309950994.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ch = ch.append(s)
/tmp/ipykernel_90747/3309950994.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ch = ch.append(s)
/tmp/ipykernel_90747/3309950994.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ch = ch.append(s)
/tmp

In [267]:
ltdt = dtxt(raw.index[-1])['day1']
sp00 = ((raw['GSPC_close'].iloc[-1] / raw.loc['2000', 'GSPC_close'].iloc[0]) - 1) * 100
sp00t = value_text(sp00)
sp18 = ((raw['GSPC_close'].iloc[-1] / raw.loc['2018', 'GSPC_close'].iloc[0]) - 1) * 100
sp18t = value_text(sp18)
text = (f'As of {ltdt}, the S\&P 500 has {sp00t} since 2000, and {sp18t} since 2018.')
write_txt(text_dir / 'eq_sp500.txt', text)  
print(text)
ns00 = ((raw['IXIC_close'].iloc[-1] / raw.loc['2000', 'IXIC_close'].iloc[0]) - 1) * 100
ns00t = value_text(ns00)
ns18 = ((raw['IXIC_close'].iloc[-1] / raw.loc['2018', 'IXIC_close'].iloc[0]) - 1) * 100
ns18t = value_text(ns18)
text = (f'The Nasdaq index {ns00t} since 2000, and {ns18t} since 2018.')
write_txt(text_dir / 'eq_nasdaq.txt', text)  
print(text)
dow00 = ((raw['DJI_close'].iloc[-1] / raw.loc['2000', 'DJI_close'].iloc[0]) - 1) * 100
dow00t = value_text(dow00)
dow18 = ((raw['DJI_close'].iloc[-1] / raw.loc['2018', 'DJI_close'].iloc[0]) - 1) * 100
dow18t = value_text(dow18)
text = (f'{dow00t} since 2000 and {dow18t} since 2018. ')
write_txt(text_dir / 'eq_dow.txt', text)  
print(text)
ru00 = ((raw['RUT_close'].iloc[-1] / raw.loc['2000', 'RUT_close'].iloc[0]) - 1) * 100
ru00t = value_text(ru00)
ru18 = ((raw['RUT_close'].iloc[-1] / raw.loc['2018', 'RUT_close'].iloc[0]) - 1) * 100
ru18t = value_text(ru18)
text = (f'Since 2000, the Russell 3000 has {ru00t}. Since 2018, the measure {ru18t}. ')
write_txt(text_dir / 'eq_russell.txt', text)  
print(text)

As of March 24, 2022, the S\&P 500 has increased 210.6 percent since 2000, and increased 67.7 percent since 2018.
The Nasdaq index increased 243.5 percent since 2000, and increased 102.5 percent since 2018.
increased 205.6 percent since 2000 and increased 39.8 percent since 2018. 
Since 2000, the Russell 3000 has increased 318.1 percent. Since 2018, the measure increased 33.9 percent. 


### Interest Rates Data From Fed

In [268]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=H15&series=4216503bb3a25c994952047659b79297&lastobs=&'+
       'from=01/01/1988&to=12/31/2022&filetype=csv&label=include&'+
       'layout=seriescolumn')
d, df = clean_fed_data(url)
df.to_csv(data_dir / 'fed_rates_raw.csv', index_label='date')

### Interest rates

In [269]:
clean_data = pd.read_csv(data_dir / 'fed_rates_raw.csv', 
                         index_col='date', parse_dates=True)

n = {'RIFLGFCY10_N.B': 'Ten-year',
    'RIFLGFCY30_N.B': 'Thirty-year',
    'RIFSPFF_N.B': 'Fed Funds',
    'RIFLGFCM03_N.B': 'Three-month',
    'RIFLGFCY05_N.B': 'Five-year',
    'RIFLGFCY02_N.B': 'Two-year',
    'RIFLGFCM01_N.B': 'One-month',
    'RIFLGFCY01_N.B': 'One-year',
    'RIFLGFCY20_N.B': 'Twenty-year',
    'RIFLGFCM06_N.B': 'Six-month',
    'RIFLGFCY03_N.B': 'Three-year',
    'RIFLGFCY07_N.B': 'Seven-year'}

df = clean_data[n.keys()].rename(n, axis=1).dropna(subset=['Ten-year'])
df.to_csv(data_dir / 'treas_raw.csv', index_label='date')

In [3]:
# Taylor Rule suggested Fed Funds rate
p = (pd.read_csv(data_dir / 'pce_pi.csv', parse_dates=['date'])
       .set_index('date')['CORE']).resample('QS').mean() * 0.5

y = ((nipa_df(retrieve_table('T10106')['Data'], ['A191RX']))
      .loc['1989':, 'A191RX'])

y_p = fred_df('GDPPOT')['VALUE'] * 1_000

o = (y - (y_p)).divide(y_p).dropna()

taylor_ff = (p + 3 + (0.5*(p - 2)) + (1*(o*100))).dropna()

taylor_ff.name = 'Value'

taylor_ff.to_csv(data_dir / 'taylor.csv', index_label='date')

In [4]:
df = pd.read_csv(data_dir / 'treas_raw.csv', index_col='date', 
                 parse_dates=True).loc['1989':]

data = (df.resample('MS').mean().iloc[:-1].append(df.iloc[-1]))
data.to_csv(data_dir / 'rates.csv', index_label='date', float_format='%g')

ldate = dtxt(data.index[-1])['day1']
tenlt = df['Ten-year'].iloc[-1]
tenpr = df['Ten-year'].iloc[-252]
twolt = df['Two-year'].iloc[-1]
twopr = df['Two-year'].iloc[-252]
val89 = df['Ten-year'].loc['1989'].mean()
lowval = data['Ten-year'].min()
lowmon = dtxt(data['Ten-year'].idxmin())['mon1']
cline10 = c_line('blue!70!black')
cline2 = c_line('cyan!90!white')
ch2yr = value_text(df['Two-year'].diff(252).iloc[-1], 
                   'increase_by', ptype='pp')
ff = fred_df('FEDTARMD')['VALUE']
ffd = ff.diff(2).iloc[-1]
fv = 'raise' if ffd > 0.3 else 'lower' if ffd <0.3 else 'maintain'
cline = c_line('blue!60!black')
text = ('United States Treasury securities, or \\textbf{treasuries}, are the '+
        'asset created by federal government borrowing. The treasuries '+
        'market is traditionally considered both very low risk and fairly '+
        'liquid. The yield on these securities has fallen over time, '+
        'from an average 10-year treasury bond annual yield of '+
        f'{val89:.1f} percent in 1989 to an average of {lowval:.2f} '+
        f'percent in {lowmon}. As of {ldate}, the constant maturity yield '+
        f'for 10-year treasury bonds is {tenlt} percent {cline10}, '+
        f'compared to {tenpr:.2f} percent one year prior. \n\nShorter-'+
        'duration treasury yield have also fallen since 1989, '+
        f'though shorter-duration treasuries are more acutely affected '+
        'by changes in the key interest rate set by the Federal Reserve. '+
        f'Over the past year, two-year treasury yields have {ch2yr}, '+
        f'as the Federal Reserve is expected to {fv} interest rates. '+
        f'As of {ldate}, the annual yield on two-year treasuries is '+
        f'{twolt} percent {cline}. ')
write_txt(text_dir / 'rates_basic.txt', text)
print(text, '\n')
val = f'{data["Fed Funds"].iloc[-1]:.2f} percent'
text = (f'The effective fed funds rate is {val}, as of {ldate} {cline}.')
write_txt(text_dir / 'rates_ff.txt', text)
print(text)

rows = ['One-month', 'Three-month', 'Six-month', 'One-year', 
        'Two-year', 'Three-year', 'Five-year', 'Seven-year', 
        'Ten-year', 'Twenty-year', 'Thirty-year']
columns = [-1, -2, -5]
data2 = df[rows].iloc[columns].T
data2.columns = [dtxt(i)['day2'] for i in data2.keys()]
curmo = pd.to_datetime(f"{dtxt(df.index[-1])['mon5']}-01")
prmo = (curmo - pd.DateOffset(months=1))
prmov = dtxt(prmo)['mon5']
prmot = dtxt(prmo)['mon2']
data2[prmot] = df.loc[prmov].mean()
pryr = (curmo - pd.DateOffset(years=1))
pryrv = dtxt(pryr)['mon5']
pryrt = dtxt(pryr)['mon2']
data2[pryrt] = df.loc[pryrv].mean()
data2['2019'] = df.loc['2019'].mean()
data2['2010 --`13'] = df.loc['2010': '2013'].mean()
data2['1998 --`00'] = df.loc['1998': '2000'].mean()
data2['1989'] = df.loc['1989'].mean()
(data2.applymap('{:,.2f}'.format).replace('nan', '--')
      .to_csv(data_dir / 'treasury_rates.tex', sep='&', 
           line_terminator='\\\ ', quotechar=' '))

sep = ff.loc[str(pd.to_datetime('today').year):]
sep.index = sep.index + pd.DateOffset(months=6)
sep.to_csv(data_dir / 'sep.csv', index_label='date', header=True)
dt = dtxt(data.index[-1])['datetime']
text = (f'\draw [dashed, black!16] (axis cs:{{{dt}}},'+
        '\pgfkeysvalueof{/pgfplots/ymin}) -- '+
        f'(axis cs:{{{dt}}}, \pgfkeysvalueof{{'+
        '/pgfplots/ymax});')
write_txt(text_dir / 'ff_proj_bar.txt', text)

url = ('https://api.stlouisfed.org/fred/series?'+
       f'series_id=FEDTARMD&api_key={fred_key}&file_type=json')
r = requests.get(url)
mdt = dtxt(pd.to_datetime(r.json()['seriess'][0]['last_updated']))['day1']

tffdt = dtxt(taylor_ff.index[-1])['qtr2']
tfflt = taylor_ff.iloc[-1]
fflt = data['Fed Funds'].iloc[-1]
tffdiff = tfflt - fflt
difftxt = value_text(tffdiff, 'above_below', ptype='pp', digits=2)
url = 'https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm'
sepcol = 'purple'
text = (f'As of {tffdt}, the modified Taylor rule suggests a federal '+
        f'funds rate of {tfflt:.1f} percent, {difftxt} the current '+
        f'rate.\n\nFOMC meeting participants provide \href{{{url}}}'+
        '{projections} which can be used to summarize policymaker '+
        'views on the future path of the federal funds rate, as seen '+
        f'by the people who set it. As of {mdt}, the median projected '+
        f'federal funds rate rate is {sep.iloc[-3]} percent for '+
        f'{sep.index[-3].year}, {sep.iloc[-2]} percent for '+
        f'{sep.index[-2].year}, and {sep.iloc[-1]} percent for '+
        f'{sep.index[-1].year} {c_box(sepcol)}.')
write_txt(text_dir / 'rates_ff_proj.txt', text)
print(text)

text = ('\\node[text width=7.8cm, anchor=west, fill=white] at (axis '+
        'description cs: 0.01, 0.08){\scriptsize Summary of Economic '+
        f'Projections (\color{{{sepcol}}}\\textbf{{SEP}}\\normalcolor) '+
        f'as of {mdt}}};')
write_txt(text_dir / 'rates_ff_proj_date.txt', text)

node = (end_node(sep, sepcol, date='y', full_year=True, anchor='south', 
               align='center', colon=False, percent=True, offset=0.15))
write_txt(text_dir / 'rates_ff_proj_node.txt', node)

/tmp/ipykernel_97777/3570789723.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = (df.resample('MS').mean().iloc[:-1].append(df.iloc[-1]))


United States Treasury securities, or \textbf{treasuries}, are the asset created by federal government borrowing. The treasuries market is traditionally considered both very low risk and fairly liquid. The yield on these securities has fallen over time, from an average 10-year treasury bond annual yield of 8.5 percent in 1989 to an average of 0.62 percent in July 2020. As of March 23, 2022, the constant maturity yield for 10-year treasury bonds is 2.32 percent (see {\color{blue!70!black}\textbf{---}}), compared to 1.62 percent one year prior. 

Shorter-duration treasury yield have also fallen since 1989, though shorter-duration treasuries are more acutely affected by changes in the key interest rate set by the Federal Reserve. Over the past year, two-year treasury yields have increased by two percentage points, as the Federal Reserve is expected to raise interest rates. As of March 23, 2022, the annual yield on two-year treasuries is 2.13 percent (see {\color{blue!60!black}\textbf{---}

### Yield Curve

In [272]:
df = pd.read_csv(data_dir / 'treas_raw.csv', index_col='date', 
                 parse_dates=True)

i = {'One-month': 1, 'Three-month': 2, 'One-year': 3, 'Two-year': 4, 
     'Five-year': 5, 'Ten-year': 6, 'Twenty-year': 7, 'Thirty-year': 8}
tbl = pd.DataFrame()
for v, c in [(-1, 'value'), (-252, 'oneyear'), (-252*5, 'fiveyear')]:
    col = df[i.keys()].iloc[v]
    col.index = col.index.map(i)
    tbl[c] = col
tbl.index.name = 'number'
tbl['alignment'] = 270

tbl.to_csv(data_dir / 'yc.csv', float_format='%g')
dt = dtxt(df.index[-1])['day1']
date = ('\\node[text width=3.8cm, anchor=west] at (axis description cs: '+
        f'0, 0.95) {{\small As of {{{dt}}}:}};')
write_txt(text_dir / 'yc_date.txt', date)
#print(date)

### Spread

In [273]:
df = pd.read_csv(data_dir / 'treas_raw.csv', index_col='date', 
                 parse_dates=True)

data = df[df['Ten-year'] != 'ND'].astype('float')
ldate = dtxt(data.index[-1])['day1']
spread = pd.DataFrame()
spread['Ten-3M'] = data['Ten-year'] - data['Three-month']
spread['Ten-2Y'] = data['Ten-year'] - data['Two-year']
spread.loc['2017':].to_csv(data_dir / 'spread.csv', index_label='date', 
                           float_format='%g', header=True)

col103 = 'blue!70!cyan!80!white'
col102 = 'red!60!violet!90!white'

node = end_node(spread['Ten-3M'], col103, digits=2, date='d', full_year=True)
write_txt(text_dir / 'spread_node.txt', node)

node = end_node(spread['Ten-2Y'], col102, digits=2, date='d', full_year=True)
write_txt(text_dir / 'spread_node2.txt', node)

def pp(value):
    return 'percentage points' if value > 1.01 else 'percentage point'

lt3 = spread['Ten-3M'].iloc[-1]
lt3t = f'{lt3:.2f} {pp(lt3)}'
pr3 = spread['Ten-3M'].iloc[-252]
pr3t = f'{pr3:.2f} {pp(pr3)}'
lt2 = spread['Ten-2Y'].iloc[-1]
lt2t = f'{lt2:.2f} {pp(lt2)}'
pr2 = spread['Ten-2Y'].iloc[-252]
pr2t = f'{pr2:.2f} {pp(pr2)}'

text = (f'As of {ldate}, the spread between a 10-year treasury bond and '+
        f'a three-month treasury bill is {lt3t} {c_line(col103)}, compared '+
        f'to {pr3t} one year prior. The spread between 10-year and '+
        f'2-year treasuries {c_line(col102)} is {lt2t} on {ldate}, and '+
        f'{pr2t} one year prior.')
write_txt(text_dir / 'spread_basic.txt', text)
print(text)

As of March 23, 2022, the spread between a 10-year treasury bond and a three-month treasury bill is 1.82 percentage points (see {\color{blue!70!cyan!80!white}\textbf{---}}), compared to 1.60 percentage points one year prior. The spread between 10-year and 2-year treasuries (see {\color{red!60!violet!90!white}\textbf{---}}) is 0.19 percentage point on March 23, 2022, and 1.48 percentage points one year prior.


In [274]:
sp = spread.loc[(spread['Ten-3M'] < 0) & 
                (spread['Ten-3M'].shift().rolling(252).min() >= 0), 
                'Ten-3M']
num_ic = len(sp) + 1 # Add one for 1989
tnum_ic = numbers[f'{num_ic:.1f}']
icdt = dtxt(sp.index[-1])['day1']

#rec = fred_df('USREC')
#num_rec = len(rec[(rec.VALUE==1) & (rec.VALUE.shift(1) == 0)])
#tnum_rec = numbers[f'{num_rec:.1f}']
tnum_rec = 'four'
text = (f'Since 1989, the US has entered into {tnum_rec} '+
        'recessions and the 10-year to 2-year segment of '+
        f'the yield curve has newly inverted {tnum_ic} times. '+
        f'The most recent such inversion started on {icdt}.')
write_txt(text_dir / 'yc_inversion.txt', text)
print(text)

Since 1989, the US has entered into four recessions and the 10-year to 2-year segment of the yield curve has newly inverted six times. The most recent such inversion started on March 22, 2019.


### Gold Price

In [275]:
date = dtxt(pd.to_datetime('today'))['datetime']
url = ('https://prices.lbma.org.uk/export/xls/?c={"metals":["gold"],'+
       '"type":"daily","currency":["usd"],"published":["am"],'+
       f'"dates":{{"start":"1989-01-01","end":"{date}"}}}}')
data = pd.read_excel(url, header=1, index_col=0, 
                     parse_dates=True).sort_index()
print('Latest Data:', dtxt(data.index[-1])['day1'])
data.to_csv(data_dir / 'gold_raw.csv', index_label='date')

Latest Data: March 24, 2022


In [276]:
data = pd.read_csv(data_dir / 'gold_raw.csv', index_col='date', 
                   parse_dates=True)
df = data.resample('MS').mean().iloc[:-1]
df.index = df.index + pd.DateOffset(days=14)
lt = data.iloc[-1]
df = df.append(lt)
df.to_csv(data_dir / 'gold.csv', index_label='date')

ltdate = dtxt(df.index[-1])['day1']
ltval = df.AM.iloc[-1]
prdate = dtxt(df.index[-13])['mon1']
prval = df.AM.iloc[-13]
grdate = dtxt(df.loc['2006':'2011', 'AM'].idxmax())['mon1']
grval = df.loc['2006':'2011', 'AM'].max()

color = 'orange!40!yellow'
node = end_node(df.AM, color, dollar=True, digits='comma', date='d', 
                offset=-0.35, full_year=True)
write_txt(text_dir / 'gold_node.txt', node)

url = 'https://www.lbma.org.uk/prices-and-data/precious-metal-prices#/table'
maxdt = df.AM.idxmax()
maxdtt = dtxt(maxdt)['mon1']
maxval = df.AM.max()
text = (f'As of {ltdate}, one troy ounce of \\textbf{{gold}} '+
        f'\href{{{url}}}{{sells}} for \${ltval:,.2f} {c_line(color)}, '
        f'compared to an average of \${prval:,.2f} per ounce one '+
        f'year prior, during {prdate}. Following the great recession, '+
        f'the monthly average price of gold reached \${grval:,.2f} '+
        f'per ounce, in {grdate}. In {maxdtt}, the average monthly '+
        f'price reached \${maxval:,.2f} per ounce.')
print(text)
write_txt(text_dir / 'gold.txt', text)

As of March 24, 2022, one troy ounce of \textbf{gold} \href{https://www.lbma.org.uk/prices-and-data/precious-metal-prices#/table}{sells} for \$1,945.90 (see {\color{orange!40!yellow}\textbf{---}}), compared to an average of \$1,721.56 per ounce one year prior, during March 2021. Following the great recession, the monthly average price of gold reached \$1,780.65 per ounce, in September 2011. In August 2020, the average monthly price reached \$1,971.17 per ounce.


/tmp/ipykernel_90747/1549099801.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lt)


### Fed Balance Sheet

In [277]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H41&series=38d757c01cb3b550176f371352643679&lastobs=&'
dt = 'from=01/01/2002&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

raw_data = pd.read_csv(url)

d = {v: re.sub("\s+[\(\[].*?[\)\]]", "", i.split(';')[0]) 
     for i, v in raw_data.iloc[4, 1:].iteritems()}

date_column = raw_data.loc[5:, 'Series Description']
date_index = pd.to_datetime(date_column).rename('Date')
columns = raw_data.iloc[4, 1:].values
    
clean_data = raw_data.iloc[5:, 1:].replace('ND', np.nan).astype('float')
clean_data.index = date_index
clean_data.columns = columns

clean_data = clean_data / 1_000_000

clean_data.to_csv(data_dir / 'fed_assets.csv', index_label='date', float_format='%g')

In [278]:
ltval = clean_data['RESPPMA_N.WW'].iloc[-1].round(1)
ltdate = dtxt(clean_data.index[-1])['day1']
n = (clean_data.iloc[-1] - clean_data.iloc[-2])['RESPPMA_N.WW'] * 1000
chval = (f'increased by \${abs(n):.0f} billion' if n >= 10 
         else f'decreased by \${abs(n):.0f} billion' 
         if n <= -10 else 'was largely unchanged')

text = ('In response to the collapse of the housing bubble, the Fed purchased '+
        'U.S. Treasury bonds and mortgage-backed securities. Total assets held by '+
        'the Federal Reserve (see {\color{blue!80!black}\\textbf{---}}) '+
        'increased from \$0.9 trillion in August 2008 to '+
        '\$2.2 trillion in November 2008. Additional rounds of asset purchases, '+
        'referred to as quantitative easing, increased the balance sheet to '+
        '\$4.5 trillion by January 2014. As bonds mature they were replaced '+
        'until October 2017, when the Fed allowed the size of its balance '+
        'sheet to normalize. Total assets fell below \$3.8 trillion in August 2019.')
write_txt(text_dir / 'fed_assets1.txt', text)
        
txt2 = ('Balance sheet normalization ended in September 2019 when the Fed increased '+
        'operations in overnight and term repurchase agreement (repo) markets, following '+
        'a sharp increase in rates in these markets. The Fed balance sheet '+
        'increased to \$4.1 trillion by December 2019.')
write_txt(text_dir / 'fed_assets2.txt', txt2)
        
txt3 = ('During the COVID-19 pandemic, the Fed offered lending to businesses and '+
        'currency swaps to major US trading partners, began to purchase commercial bonds, '+
        'and expanded purchases of treasuries and mortgage-backed securities.\n\n '+
        'The Fed balance sheet increased from \$4.2 trillion in February 2020 to '+
        f'\${ltval} trillion, as of the latest data, covering {ltdate}. ')
write_txt(text_dir / 'fed_assets3.txt', txt3)
print(txt3)
        
### Add something about recent pace (e.g. "The total value of Fed assets {chval} from the value one week prior.")

During the COVID-19 pandemic, the Fed offered lending to businesses and currency swaps to major US trading partners, began to purchase commercial bonds, and expanded purchases of treasuries and mortgage-backed securities.

 The Fed balance sheet increased from \$4.2 trillion in February 2020 to \$9.0 trillion, as of the latest data, covering March 23, 2022. 


### Fed Balance Sheet Table

In [279]:
base = 'https://www.federalreserve.gov/datadownload/Output.aspx?'
srs = 'rel=H41&series=bbb7b50f663fe933c8cce7e3707e3998&lastobs=&'
dt = 'from=01/01/2002&to=12/31/2022&'
oth = 'filetype=csv&label=include&layout=seriescolumn'
url = base + srs + dt + oth

d, clean_data = clean_fed_data(url)

d2 = {'RESPPA_N.WW': '\\textbf{Total} (see {\color{blue!80!black}\\textbf{---}})',
      'RESPPALGUO_N.WW': '\hspace{2mm}U.S. Treasury securities',
      'RESPPALGASMO_N.WW': '\hspace{2mm}Mortgage-backed securities',
      'RESH4SCS_N.WW': '\hspace{2mm}Central bank liquidity swaps',
      'RESPPALGTR_N.WW': '\hspace{2mm}Repurchase agreements',
      'RESPPALD_N.WW': '\hspace{2mm}Loans',
      'RESPPALDJ_N.WW': '\hspace{4mm}Payroll Protection Program',
      'NetPremDisc': '\hspace{2mm}Net unamortized premium',
      'Other': '\hspace{2mm}Other'}

pr_di = ['RESPPALSD_N.WW', 'RESPPALSP_N.WW']

data = clean_data.copy()
data['Other'] = (data['RESPPA_N.WW'] - 
                             data.drop('RESPPA_N.WW', axis=1).sum(axis=1))
data['NetPremDisc'] = data[pr_di].sum(axis=1)
ltval_treas = data['RESPPALGUO_N.WW'].iloc[-1] / 1_000_000
ltval_mbs = data['RESPPALGASMO_N.WW'].iloc[-1] / 1_000_000

data = data.rename(d2, axis=1).drop(pr_di, axis=1)

df = pd.DataFrame()

for i in [-1, -2, -5, -53, -105]:
    df[dtxt(data.index[i])['day2']] = data.iloc[i]
    
(df.loc[list(d2.values()), :].div(1000)
   .applymap('{:,.1f}'.format)
   .to_csv(data_dir / 'fed_bal_sheet.tex', sep='&', 
           line_terminator='\\\ ', quotechar=' '))

text = (f'The Fed currently holds \${ltval_treas:,.1f} trillion in Treasuries '+
        f'and \${ltval_mbs:,.1f} trillion in mortgage-backed securities. ')
write_txt(text_dir / 'fed_assets4.txt', text)
print(text)

The Fed currently holds \$5.8 trillion in Treasuries and \$2.7 trillion in mortgage-backed securities. 


### Exchange rates

In [280]:
url = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
       'rel=H10&series=8dea680ff61ef97f0aefa5b17d760d87&lastobs=&'+
       'from=01/01/2015&to=12/31/2022&filetype=csv&label=include&'+
       'layout=seriescolumn')
d, clean_data = clean_fed_data(url)
clean_data.to_csv(data_dir / 'fx_raw_st.csv', index_label='date')

url2 = ('https://www.federalreserve.gov/datadownload/Output.aspx?'+
        'rel=H10&series=3b4f0209725fa1526861cfa9eeea0473&lastobs=&'+
        'from=01/01/1989&to=12/31/2022&filetype=csv&label=include&'+
        'layout=seriescolumn')
d, clean_data = clean_fed_data(url2)
clean_data.to_csv(data_dir / 'fx_raw_lt.csv', index_label='date')

In [281]:
clean_data = pd.read_csv(data_dir / 'fx_raw_lt.csv', index_col='date', 
                         parse_dates=True)

for cc in ['EU', 'UK']:
    clean_data[f'RXI_N.B.{cc}'] = 1 / clean_data[f'RXI$US_N.B.{cc}'] 
clean_data['RXI_N.B.JA'] = clean_data['RXI_N.B.JA'] / 100.0

latest = clean_data.dropna(how='all').iloc[-1]
major = ['RXI_N.B.EU', 'RXI_N.B.UK', 'RXI_N.B.CA', 'RXI_N.B.JA']
indx = ['JRXWTFB_N.B', 'JRXWTFN_N.B', 'JRXWTFO_N.B']
twidx = (clean_data.resample('MS').mean()
                   .append(latest)[indx].dropna())
node = end_node(twidx['JRXWTFB_N.B'], 'blue!60!black', date='day', 
                full_year=True)
write_txt(text_dir / 'twd_node.txt', node)
adj = node_adj(twidx)
smax = twidx.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

cols = {'JRXWTFN_N.B': 'violet!90!blue', 
        'JRXWTFO_N.B': 'green!75!yellow!90!black'}
date = {series: 'd' if series == smax else None 
        for series in cols.keys()}
nodes  ='\n'.join([end_node(twidx[series], color, 
                            date=date[series], 
                            full_year=True, digits=2,
                            size=1.1, offset=adj[series]) 
                   for series, color in cols.items()])
write_txt(text_dir / 'twd_nodes.txt', nodes) 
twidx.to_csv(data_dir / 'fx_idx.csv', index_label='date', 
             float_format='%g', header=True)

# Advanced and emerging markets index txt
ltadv = value_text(latest['JRXWTFN_N.B'] - 100)
ltem = value_text(latest['JRXWTFO_N.B'] - 100)
text = ('The Fed separately calculates the trade-weighted exchange '+
        'rate with \\textbf{advanced economies}, and with '+
        '\\textbf{emerging markets}. Since 2006, the dollar has '+
        f'{ltem} against emerging market currencies '+
        f'{c_line(cols["JRXWTFO_N.B"])}, and {ltadv} against advanced '+
        f'economy currencies {c_line(cols["JRXWTFN_N.B"])}. ')
write_txt(text_dir / 'twd_adv_em.txt', text)
print(text, '\n')

(clean_data.resample('MS').mean().append(latest)[major]
           .to_csv(data_dir / 'fx1.csv', index_label='date', 
                   float_format='%g'))
df = clean_data[major]
df.columns = [i[-2:] for i in df.columns]
adj = node_adj(df)
smax = df.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

cols = {'CA': 'green!85!blue', 'JA': 'red', 'EU': 
        'cyan!90!white', 'UK': 'blue!90!cyan'}
date = {series: 'd' if series == smax else None 
        for series in cols.keys()}
nodes  ='\n'.join([end_node(df[series], color, 
                            date=date[series], 
                            full_year=True, digits=2,
                            size=1.1, offset=adj[series]) 
                   for series, color in cols.items()])
write_txt(text_dir / 'fx_nodes.txt', nodes) 

# Broad dollar index text
ldate = dtxt(twidx.index[-1])['day1']
lval = twidx['JRXWTFB_N.B'].iloc[-1]
totch = ((lval / 100) - 1) * 100
threeyr = twidx['JRXWTFB_N.B'].iloc[-38:].mean()
prev3yr = twidx['JRXWTFB_N.B'].iloc[-74:-38].mean()
text = (f'As of {ldate}, the broad dollar index is {totch:.1f} '+
        'percent above its value at inception in '+
        f'2006. Over the past three years, the index value has '+
        f'averaged {threeyr:.1f}, compared to an average of '+
        f'{prev3yr:.1f} over the previous three-years.')
write_txt(text_dir / 'twdbasic.txt', text)
print(text, '\n')

cl = {name: c_line(col) for name, col in cols.items()}
lt = clean_data.loc[ldate]
ltd = {name: lt[f'RXI_N.B.{name}'] for name in cols.keys()}
pc = clean_data.pct_change(262).iloc[-1] * 100
pcd = {name: value_text(pc[f'RXI_N.B.{name}'], threshold=0.1) for name in cols.keys()}
text = (f'As of {ldate}, one US dollar buys approximately: '+
        f'{ltd["CA"]:.2f} Canadian dollars {cl["CA"]}, '+
        f'{ltd["JA"] * 100:.0f} Japanese yen {cl["JA"]}, '+
        f'{ltd["EU"]:.2f} euros {cl["EU"]}, and {ltd["UK"]:.2f} '+
        f'British pounds {cl["UK"]}. Over the past three years, '+
        f'the nominal exchange rate between the US dollar and '+
        f'the Canadian dollar {pcd["CA"]}, the USD-JPY rate '+
        f'{pcd["JA"]}, the USD-EUR rate {pcd["EU"]}, and the '+
        f'USD-GBP rate {pcd["UK"]}.')
write_txt(text_dir / 'selcurr_basic.txt', text)
print(text)

The Fed separately calculates the trade-weighted exchange rate with \textbf{advanced economies}, and with \textbf{emerging markets}. Since 2006, the dollar has increased 24.6 percent against emerging market currencies (see {\color{green!75!yellow!90!black}\textbf{---}}), and increased 9.6 percent against advanced economy currencies (see {\color{violet!90!blue}\textbf{---}}).  

As of March 18, 2022, the broad dollar index is 16.3 percent above its value at inception in 2006. Over the past three years, the index value has averaged 115.6, compared to an average of 112.6 over the previous three-years. 

As of March 18, 2022, one US dollar buys approximately: 1.26 Canadian dollars (see {\color{green!85!blue}\textbf{---}}), 119 Japanese yen (see {\color{red}\textbf{---}}), 0.91 euros (see {\color{cyan!90!white}\textbf{---}}), and 0.76 British pounds (see {\color{blue!90!cyan}\textbf{---}}). Over the past three years, the nominal exchange rate between the US dollar and the Canadian dollar in

/tmp/ipykernel_90747/4022257957.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  twidx = (clean_data.resample('MS').mean()
/tmp/ipykernel_90747/4022257957.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  (clean_data.resample('MS').mean().append(latest)[major]


### Exchange Rates Table 

In [282]:
clean_data = pd.read_csv(data_dir / 'fx_raw_st.csv', index_col='date', 
                         parse_dates=True)
for cc in ['EU', 'UK', 'AL', 'NZ']:
    clean_data[f'RXI_N.B.{cc}'] = 1 / clean_data[f'RXI$US_N.B.{cc}'] 

fx = {'RXI_N.B.EU': 'EUR',
      'RXI_N.B.UK': 'GBP',
      'RXI_N.B.JA': 'JPY',
      'RXI_N.B.CA': 'CAD',
      'RXI_N.B.MX': 'MXN',
      'RXI_N.B.CH': 'CNY',
      'RXI_N.B.SZ': 'CHF',
      'RXI_N.B.HK': 'HKD',
      'RXI_N.B.IN': 'INR',
      'RXI_N.B.AL': 'AUD',
      'RXI_N.B.NZ': 'NZD',
      'RXI_N.B.BZ': 'BRL',
      'RXI_N.B.KO': 'KRW',
      'RXI_N.B.MA': 'MYR',
      'RXI_N.B.DN': 'DKK',
      'RXI_N.B.NO': 'NOK',
      'RXI_N.B.SD': 'SEK',
      'RXI_N.B.SF': 'ZAR',
      'RXI_N.B.SI': 'SGD',
      'RXI_N.B.TA': 'TWD'}

tbl_data = clean_data[fx.keys()].dropna(how='all')
tbl_data.columns = fx.values()
#tbl_data.loc[:,'JPY'] *= 100

table = pd.DataFrame()
table[dtxt(tbl_data.index[-1])['day2']] = tbl_data.iloc[-1]
table['1-month moving average'] = tbl_data.iloc[-22:].mean()
table['1-year moving average'] = tbl_data.iloc[-262:].mean()
table['2019 average'] = tbl_data.loc['2019'].mean()
dec1 = ['JPY', 'KRW']
table.loc[['JPY', 'KRW'],:] = table.loc[['JPY', 'KRW'],:].applymap("{0:.1f}".format)
dec3 = ['GBP', 'EUR', 'CHF', 'AUD', 'NZD', 'CAD', 'SGD']
table.loc[dec3,:] = table.loc[dec3,:].applymap("{0:.3f}".format)
table.loc[~table.index.isin(dec3+dec1)] = (table.loc[~table.index.isin(dec3+dec1)]
                                           .applymap("{0:.2f}".format))
table['1-month percent change'] = (tbl_data.pct_change(22)
                                   * 100).iloc[-1].apply("{0:.1f}".format)
table['1-year percent change'] = (tbl_data.pct_change(262)
                                  * 100).iloc[-1].apply("{0:.1f}".format)
table['5-year percent change'] = (tbl_data.pct_change(262*5)
                                  * 100).iloc[-1].apply("{0:.1f}".format)

table.index = [f'\includegraphics[width=.03\\textwidth]{{data/flags/{cc}}} \ {cc}' 
               for cc in table.index]

(table.to_csv(data_dir / 'fx_table.tex', sep='&', 
              line_terminator='\\\ ', quotechar=' '))

### Jobless claims

In [283]:
data = fred_df('ICNSA', start='2017') / 1000 
data['1M'] = data['VALUE'].rolling(4).mean()
data.div(1000).to_csv(data_dir / 'icsa.csv', index_label='date', 
                      float_format='%g')

totval = data['VALUE'].iloc[-1]*1000
datelt = dtxt(data.index[-1])['day1']
latest1m = data["1M"].iloc[-1]*1000
prev1m = data["1M"].iloc[-53]*1000

chval = totval - data['VALUE'].iloc[-2]*1000
chtxt = f'{round(abs(chval),-2):,.0f}'
if chval > 1000:
    change = f'a one-week increase of {chtxt}'
elif chval < -1000:
    change = f'a one-week decrease of {chtxt}'
else:
    change = 'virtually unchanged from the previous week'
url = 'https://www.dol.gov/ui/data.pdf'
col = 'cyan!80!blue'
text = (f'The Department of Labor \href{{{url}}}{{report}} '+
        f'{totval:,.0f} actual \\textbf{{new claims for unemployment '+
        f'insurance}} (UI) under state programs {c_line(col)} '+
        f'during the week ending {datelt}, {change}. Over '+
        f'the past four weeks, new claims have averaged '+
        f'{round(latest1m,-2):,.0f} per week. During the same '+
        f'four-week period last year, there were an average of '+
        f'{round(prev1m,-2):,.0f} new claims per week.')
write_txt(text_dir / 'icsa.txt', text)
print(text)
data = fred_df('CCNSA', start='2017') / 1000 
data.div(1000).to_csv(data_dir / 'ccsa.csv', index_label='date', 
                      float_format='%g')

totval = data['VALUE'].iloc[-1]*1000
prval = data['VALUE'].iloc[-2]*1000
datelt = dtxt(data.index[-1])['day1']
prevyrval = data['VALUE'].iloc[-53]*1000
prmoval = data['VALUE'].iloc[-4]*1000
datepr = dtxt(data.index[-53])['day1']

chval = totval - prval
chtxt = f'{round(abs(chval),-2):,.0f}'
if chval > 1000:
    change = f'a one-week increase of {chtxt}'
elif chval < -1000:
    change = f'a one-week decrease of {chtxt}'
else:
    change = 'virtually unchanged from the previous week'
color = 'green!90!blue'
text = (f'For the week ending {datelt}, the Department of Labor '+
        '\href{{https://www.dol.gov/ui/data.pdf}}{{reports}} '+
        f'{totval:,.0f} continued claims for unemployment '+
        'insurance (insured unemployed) under state programs '+
        f'{c_line(color)}, {change}. One year prior, during '+
        f'the week of {datepr}, there were an average of '+
        f'{round(prevyrval,-2):,.0f} insured unemployed.')
write_txt(text_dir / 'ccsa.txt', text)
print(text)

The Department of Labor \href{https://www.dol.gov/ui/data.pdf}{report} 181,087 actual \textbf{new claims for unemployment insurance} (UI) under state programs (see {\color{cyan!80!blue}\textbf{---}}) during the week ending March 19, 2022, a one-week decrease of 22,800. Over the past four weeks, new claims have averaged 200,200 per week. During the same four-week period last year, there were an average of 721,800 new claims per week.
For the week ending March 12, 2022, the Department of Labor \href{{https://www.dol.gov/ui/data.pdf}}{{reports}} 1,730,622 continued claims for unemployment insurance (insured unemployed) under state programs (see {\color{green!90!blue}\textbf{---}}), a one-week decrease of 73,700. One year prior, during the week of March 13, 2021, there were an average of 4,233,700 insured unemployed.


In [284]:
file = 'https://oui.doleta.gov/unemploy/docs/weekly_pandemic_claims.xlsx'
fed_raw = pd.read_excel(file, usecols='A:F', parse_dates=True).dropna()
fed_raw['pua_ic'] = (fed_raw['PUA IC'].astype(str).str.strip()
                     .replace('', '0').astype('int'))
pua_ic = fed_raw.groupby('Rptdate')['pua_ic'].sum()
pua_ic.index = pd.to_datetime(pua_ic.index)
pua_ic.index.name = 'date'
cc_raw = fed_raw.dropna(subset=['PUA CC'])
pua_cc = cc_raw.groupby('Rptdate')['PUA CC'].sum()
peuc_cc = cc_raw.groupby('Rptdate')['PEUC CC'].sum()
fed_cc = pua_cc + peuc_cc
fed_cc.index = pd.to_datetime(fed_cc.index)
fed_cc.index.name = 'date'
fed_cc.name = 'fed_cc'

d1 = pd.DataFrame([pua_ic, fed_cc]).T / 1000
d1.div(1000).to_csv(data_dir / 'fed_uic.csv', index_label='date', float_format='%g')

ltdate = dtxt(d1.index[-1])['day1']
prdatecc = dtxt(d1.dropna().index[-4])['day1']
ltdatecc = dtxt(d1.dropna().index[-1])['day1']
pua_ic_lt = f"{round(d1['pua_ic'].iloc[-1] * 1000, -2):,.0f}"
pua_ic_pr = f"{round(d1['pua_ic'].iloc[-2] * 1000, -2):,.0f}"
pua_ic_1m = f"{round(d1['pua_ic'].iloc[-4:].mean() * 1000, -2):,.0f}"
fed_cc_lt = d1.dropna()['fed_cc'].iloc[-1] * 1_000
fed_cc_mo = d1.dropna()['fed_cc'].iloc[-4] * 1_000
fed_cc_ltt = f"{fed_cc_lt:,.0f}"
tot_cc = fed_cc_lt + totval
tot_lt = f"{tot_cc / 1_000_000:,.1f} million"
tot_cc_pr = fed_cc_mo + prmoval
tot_pr = f"{tot_cc_pr / 1_000_000:,.1f} million"

text = (f'Over the week ending {ltdate}, there were {pua_ic_lt} '+
        'initial UI \href{https://oui.doleta.gov/unemploy/DataDashboard.asp}{claims} '+
        'under the Pandemic Unemployment Assistance '+
        '(PUA) program (see {\color{blue!50!purple!80!black}\\textbf{---}}), '+
        f'compared to {pua_ic_pr} during the prior week, '+
        f'and an average of {pua_ic_1m} initial claims per week over the '+
        'past four weeks. \n\n'+
        f'Federal program continuing claims total {fed_cc_ltt} '+
        f'in {ltdatecc} '+
        '(see {\color{green!50!blue}\\textbf{---}}). These include both ' +
        'claims under the PUA program and claims under the Pandemic Emergency '+
        'Unemployment Compensation (PEUC) program. Combining federal program '+
        f'claims with state program claims indicates there are a total '+
        f'of {tot_lt} insured unemployed persons during the week ending {ltdatecc}, '+
        f'compared to {tot_pr} one month prior, during the week ending {prdatecc}. ')
write_txt(text_dir / 'fed_uic.txt', text)
print(text)

Over the week ending March 5, 2022, there were 600 initial UI \href{https://oui.doleta.gov/unemploy/DataDashboard.asp}{claims} under the Pandemic Unemployment Assistance (PUA) program (see {\color{blue!50!purple!80!black}\textbf{---}}), compared to 500 during the prior week, and an average of 600 initial claims per week over the past four weeks. 

Federal program continuing claims total 101,041 in March 5, 2022 (see {\color{green!50!blue}\textbf{---}}). These include both claims under the PUA program and claims under the Pandemic Emergency Unemployment Compensation (PEUC) program. Combining federal program claims with state program claims indicates there are a total of 1.8 million insured unemployed persons during the week ending March 5, 2022, compared to 2.0 million one month prior, during the week ending February 12, 2022. 


### VIX (SP500 volatility)

In [285]:
VIX = ('https://cdn.cboe.com/api/global/us_indices/daily_prices/'+
       'VIX_History.csv')
curr = pd.read_csv(VIX, index_col='DATE', parse_dates=True)
df = (curr['CLOSE'].resample('MS').mean()
      .append(curr['CLOSE'].iloc[-1:]).rename('value'))
df.to_csv(data_dir / 'vix.csv', index_label='date', header='True')
color = 'magenta'
node = end_node(df, color, date='d', offset=0.35, full_year=True)
write_txt(text_dir / 'vix_node.txt', node)

ldate = dtxt(df.index[-1])['day1']
vallt = df.iloc[-1]
val3y = df.iloc[-37:].mean()

compare = compare_text(vallt, val3y, [3, 12, 30])

one_wk = (value_text(curr.CLOSE.diff(5).iloc[-1], 
                     style='increase_by', ptype='pp')
          .replace('percentage ', ''))

text = (f'This volatility measure, the VIX index {c_line(color)}, '+
        f'was {vallt:.1f} on {ldate}, {compare} the average index '+
        f'value of {val3y:.1f} over the past three years. The VIX '+
        f'{one_wk} over the past week.')
write_txt(text_dir / 'vixbasic.txt', text)
print(text)

This volatility measure, the VIX index (see {\color{magenta}\textbf{---}}), was 23.6 on March 23, 2022, in line with the average index value of 22.2 over the past three years. The VIX decreased by 3.1 points over the past week.


/tmp/ipykernel_90747/3621382974.py:4: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = (curr['CLOSE'].resample('MS').mean()


### Oil prices (WTI)

In [286]:
url = (f'https://api.eia.gov/series/?api_key={eia_key}'+
       '&series_id=PET.RCLC1.D&start=19890101')

r = requests.get(url).json()
data = (pd.Series({pd.to_datetime(f'{i[0]}'): i[1] 
                   for i in r['series'][0]['data']})
          .sort_index().to_frame(name='VALUE'))
print('Latest Data:', dtxt(data.index[-1])['day1'])
data.to_csv(data_dir / 'wti_raw.csv', index_label='date')

Latest Data: March 22, 2022


In [287]:
data = pd.read_csv(data_dir / 'wti_raw.csv', index_col='date', 
                   parse_dates=True)
df = data.resample('MS').mean().iloc[:-1]
df.index = df.index + pd.DateOffset(days=14)
lt = data.iloc[-1]
df = df.append(lt)
df.to_csv(data_dir / 'wti.csv', index_label='date')

oneyr = value_text(df.VALUE.pct_change(12).iloc[-1] * 100)
twoyr = value_text(df.VALUE.pct_change(24).iloc[-1] * 100)

color = 'red!80!purple'
node = end_node(df.VALUE, color, dollar=True, digits=2, date='d', 
                offset=True, full_year=True)
write_txt(text_dir / 'oil_node.txt', node)

maxdt = df.VALUE.idxmax()
maxdtt = dtxt(maxdt)['mon1']
maxval = df.VALUE.max()
ltch = df.loc[maxdt, 'VALUE'] - df.VALUE.iloc[-1]
ldate = dtxt(df.index[-1])['day1']
url = 'https://www.eia.gov/dnav/pet/hist/RCLC1D.htm'
text = (f'As of {ldate}, the benchmark \href{{{url}}}{{futures price}} '+
        f'for a barrel of west Texas intermediate (WTI) '+
        f'\\textbf{{crude oil}} is \${lt.VALUE:.2f} {c_line(color)}. '+
        f'Over the past year, this measure of oil prices {oneyr}. Over '+
        f'the past two years, the price {twoyr}. The WTI price is '+
        f'currently \${ltch:.0f} per barrel below its peak monthly '+
        f'average price of \${maxval:.0f} per barrel in {maxdtt}.')
write_txt(text_dir / 'wti.txt', text)
print(text)

As of March 22, 2022, the benchmark \href{https://www.eia.gov/dnav/pet/hist/RCLC1D.htm}{futures price} for a barrel of west Texas intermediate (WTI) \textbf{crude oil} is \$111.76 (see {\color{red!80!purple}\textbf{---}}). Over the past year, this measure of oil prices increased 79.2 percent. Over the past two years, the price increased 267.1 percent. The WTI price is currently \$22 per barrel below its peak monthly average price of \$134 per barrel in June 2008.


/tmp/ipykernel_90747/2215678221.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lt)


### Inflation Expectations

In [288]:
data1 = fred_df('T5YIE').loc['2015':,'VALUE']
data1.to_csv(data_dir / 'infbreak.csv', index_label='date', 
             header=True)
data2 = fred_df('T5YIFR').loc['2015':,'VALUE']
df = pd.DataFrame({'5_year_breakeven': data1, 
                   '5_year_5_year_forward': data2})
df.to_csv(data_dir / 'infbreak_comb.csv', index_label='date')

color = 'blue!70!black'

node = end_node(data1, color, percent=True)
write_txt(text_dir / 'infbreak_node.txt', node)

ldatem = dtxt(data1.index[-1])['day1']
lvalm = data1.iloc[-1]
pdatem = dtxt(data1.dropna().index[-252])['day1']
pvalm = data1.dropna().iloc[-252]
p5valm = data1.dropna().iloc[-(252*5)]

text = (f'As of {ldatem}, markets expect an average inflation '+
        f'rate of {lvalm:.1f} percent over the next five ' + 
        f'years {c_line(color)}, compared to an expected rate '+
        f'of {pvalm:.1f} percent on {pdatem}. Markets had expected '+
        f'inflation to average {p5valm:.1f} percent per year over '+
        f'the past five years, five years ago.')
write_txt(text_dir / 'inf_exp_mkts.txt', text)
print(text)

p55val = data2.iloc[-1]
if data2.iloc[-1] + 0.1 > data1.iloc[-1]:
    compare = 'fall below '
elif data2.iloc[-1] - 0.1 < data1.iloc[-1]:
    compare = 'exceed '
else:
    compare = 'maintain the same rate as '
text = (f'Over this five-year period, markets suggest {p55val:.1f} '+
        f'percent inflation per year. Inflation rates in the near-term '+
        f'are therefore expected to {compare}inflation rates in the '+
        'longer-term')
write_txt(text_dir / 'inf_exp_mkts_55.txt', text)
print(text)

As of March 24, 2022, markets expect an average inflation rate of 3.6 percent over the next five years (see {\color{blue!70!black}\textbf{---}}), compared to an expected rate of 2.5 percent on March 25, 2021. Markets had expected inflation to average 1.9 percent per year over the past five years, five years ago.
Over this five-year period, markets suggest 2.3 percent inflation per year. Inflation rates in the near-term are therefore expected to exceed inflation rates in the longer-term


### High Yield Corporate Bond

In [289]:
df = fred_df('BAMLH0A0HYM2EY')
data = df.dropna().resample('MS').mean().append(df.iloc[-1])
data = data[~data.index.duplicated(keep='last')]
data.to_csv(data_dir / 'highyield.csv', index_label='date')
color = 'red!50!purple'

node = end_node(data['VALUE'], color, percent=True, date='d', full_year=True)
write_txt(text_dir / 'highyield_node.txt', node)

ltdt = dtxt(df.index[-1])['day1']
ltval = data.VALUE.iloc[-1]
mm = -2 if df.index[-1].is_month_start == True else -3
prmoval = data.VALUE.iloc[mm]
prmo = dtxt(data.index[mm])['mon1']
val19 = data.loc['2019', 'VALUE'].mean()

text = (f'As of {ltdt}, the effective yield for \\textbf{{high yield '+
        f'corporate bonds}} in the index is {ltval:.1f} percent '+
        f'{c_line(color)}. In {prmo}, the average effective yield was '+
        f'{prmoval:.1f} percent. Prior to the COVID-19 pandemic, '+
        f'in 2019, the average effective yield was {val19:.1f} '+
        'percent. ')
write_txt(text_dir / 'highyield.txt', text)
print(text)

As of March 23, 2022, the effective yield for \textbf{high yield corporate bonds} in the index is 6.0 percent (see {\color{red!50!purple}\textbf{---}}). In February 2022, the average effective yield was 5.4 percent. Prior to the COVID-19 pandemic, in 2019, the average effective yield was 6.1 percent. 


/tmp/ipykernel_90747/2554936523.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = df.dropna().resample('MS').mean().append(df.iloc[-1])


### Corporate Bonds Total Returns Index

In [290]:
#df = fred_df('BAMLCC0A0CMTRIV')
#data = df.dropna().resample('MS').mean().append(df.iloc[-1])
#data = data[~data.index.duplicated(keep='last')]
#data.to_csv(data_dir / 'corpbond_tri.csv', index_label='date')
#color = 'violet!80!blue'

#node = end_node(data['VALUE'], color, percent=True, date='d', full_year=True)
#write_txt(text_dir / 'cbtri_node.txt', node)

### Interest Rates Decomposition

In [291]:
url = ('https://www.clevelandfed.org/en/our-research/indicators-and-data/'+
       '~/media/content/our%20research/indicators%20and%20data/'+
       'inflation%20expectations/ie%20latest/ie%20xls.xls?la=en')
df = pd.read_excel(url, sheet_name='Expected Inflation', index_col=0, 
                   parse_dates=True)
df.columns = df.columns.str.replace('Expected Inflation', '').str.strip()
df.to_csv(data_dir / 'exp_infl_raw.csv', index_label='date')

df = pd.read_excel(url, sheet_name='Real Interest Rate', index_col=0, 
                   parse_dates=True)
df.columns = df.columns.str.strip()
df.to_csv(data_dir / 'frbcle_real_yield.csv', index_label='date')

In [293]:
df = pd.read_csv(data_dir / 'exp_infl_raw.csv', index_col='date', 
                  parse_dates=True) * 100
ltdt = dtxt(df.index[-1])['mon1']
write_txt(text_dir / 'frbcle_ry_date.txt', ltdt)
df2 = pd.read_csv(data_dir / 'treas_raw.csv', index_col='date', 
                  parse_dates=True).resample('MS').mean()

data, data2 = pd.DataFrame(), pd.DataFrame()
for dv in [3, 36]:
    for yr in [2, 5, 10]:
        ty = df2[f'{numbers2[yr].capitalize()}-year'].diff(dv).rename('Total')
        ie = df[f'{yr} year'].diff(dv).rename('IE')
        ry = (ty - ie).rename('RY')
        res = pd.DataFrame([ty, ie, ry]).T.iloc[-1]
        if dv == 3:
            data[f'{yr}-year'] = res
        else:
            data2[f'{yr}-year'] = res
tbl, tbl2 = data.T, data2.T
tbl.to_csv(data_dir / 'inf_exp_ch.csv', index_label='name')
tbl2.to_csv(data_dir / 'inf_exp_ch2.csv', index_label='name')
offset = 0.08
cols = ['IE', 'RY']
nodes = []
for col, row in itertools.product(cols, [0, 1, 2]):
    sdf = tbl[cols].iloc[row]
    i = tbl[col].iloc[row]
    if tbl[col].iloc[row] >= 0: 
        h = ((sdf.cumsum() - (sdf / 2) + offset))
    else:
        h = (sdf / 2) + offset
    v = h.to_dict()
    node = (f'\\absnode{{{row}.22}}{{{v[col]}}}'+
            f'{{\scriptsize {i:.2f}}}')
    nodes.append(node)
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'inf_exp_ch_nodes.txt', nodetext)
offset = 0.12
nodes = []
for col, row in itertools.product(cols, [0, 1, 2]):
    sdf = tbl2[cols].iloc[row]
    i = tbl2[col].iloc[row]
    if tbl2[col].iloc[row] >= 0: 
        h = ((sdf.cumsum() - (sdf / 2) + offset))
    else:
        h = (sdf / 2) + offset
    v = h.to_dict()
    node = (f'\\absnode{{{row}.22}}{{{v[col]}}}'+
            f'{{\scriptsize {i:.2f}}}')
    nodes.append(node)
nodetext = '\n'.join(nodes)
write_txt(text_dir / 'inf_exp_ch_nodes2.txt', nodetext)

tx = pd.DataFrame({col: tbl[col].apply(lambda x: value_text(x, ptype='pp', digits=2)) 
                   for col in tbl.columns})
tx2 = tx.loc['2-year']
tx10 = tx.loc['10-year']
tx_ = pd.DataFrame({col: tbl2[col].apply(lambda x: value_text(x, ptype='pp', digits=2)) 
                   for col in tbl2.columns})
tx_2 = tx_.loc['2-year']
tx_10 = tx_.loc['10-year']


text = (f'Over the three months ending {ltdt}, nominal two-year '+
        f'treasury yields {tx2.Total}, real yields {tx2.RY}, and '+
        f'inflation expectations {tx2.IE}. Ten-year treasury nominal '+
        f'yields {tx10.Total}, real yields {tx10.RY}, and inflation '+
        f'expectations {tx10.IE}.\n\nOver the three years ending {ltdt}, '+
        f'the nominal yield on two-year treasuries {tx_2.Total}, '+
        f'the real yield {tx_2.RY}, and inflation expectations {tx_2.IE}. '+
        f'For ten-year treasuries, the nominal yield {tx_10.Total}, '+
        f'the real yield {tx10.RY}, and expected inflation {tx_10.IE}.')
write_txt(text_dir / 'real_yield_model_ch.txt', text) 
print(text)

Over the three months ending March 2022, nominal two-year treasury yields increased 1.10 percentage points, real yields increased 0.81 percentage point, and inflation expectations increased 0.28 percentage point. Ten-year treasury nominal yields increased 0.57 percentage point, real yields increased 0.40 percentage point, and inflation expectations increased 0.17 percentage point.

Over the three years ending March 2022, the nominal yield on two-year treasuries decreased 0.63 percentage point, the real yield decreased 1.47 percentage points, and inflation expectations increased 0.83 percentage point. For ten-year treasuries, the nominal yield decreased 0.54 percentage point, the real yield increased 0.40 percentage point, and expected inflation increased 0.24 percentage point.


### Real Interest Rates

In [294]:
df = pd.read_csv(data_dir / 'frbcle_real_yield.csv', index_col='date', 
                  parse_dates=True) * 100
clean_data = pd.read_csv(data_dir / 'fed_rates_raw.csv', 
                         index_col='date', parse_dates=True)

n = {'RIFLGFCY10_XII_N.B': 'Ten-year',
     'RIFLGFCY05_XII_N.B': 'Five-year'}

df3 = (clean_data[n.keys()].rename(n, axis=1)
       .dropna(subset=['Ten-year']))

data = pd.concat([df3.resample('MS').mean().iloc[:-1], 
                  df3.iloc[-1].to_frame().T], axis=0)

#res = (pd.concat([data['Ten-year'], df], axis=1)
#         .loc['1989':])
res = df.loc['1989':]
res.to_csv(data_dir / 'real_rates2.csv', index_label='date', 
            float_format='%g')

adj = node_adj(res)
smax = res.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'Real Rate 10-year': 'blue', 
          'Real Rate 1-year': 'green!68!black'}
date = {series: 'm' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(res[series], color, 
                            date=date[series], full_year=True, 
                            percent=True,
                            digits=2, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'real_yield_nodes2.txt', nodes) 

ltdt = dtxt(res.index[-1])['mon1']
ltval = res["Real Rate 10-year"].iloc[-1]
val90 = res.loc['1990':'1999', 'Real Rate 10-year'].mean()
lt1 = res["Real Rate 1-year"].iloc[-1]
lt190 = res.loc['1990':'1999', 'Real Rate 1-year'].mean()
cl = c_line(colors["Real Rate 10-year"])
cl2 = c_line(colors["Real Rate 1-year"])
text = ('The model-based real yield on ten-year treasuries is '+
        f'{ltval:.2f} percent, as of {ltdt} {cl}. Ten-year '+
        f'treasury real yields averaged {val90:.2f} percent during '+
        'the 1990s. The model-based real yield for one-year '+
        f'treasuries is {lt1:.2f} percent in {ltdt}, compared '+
        f'to an average of {lt190:.2f} percent during the 1990s '+
        f'{cl2}. ')
write_txt(text_dir / 'real_yield_model.txt', text) 
print(text)

The model-based real yield on ten-year treasuries is 0.31 percent, as of March 2022 (see {\color{blue}\textbf{---}}). Ten-year treasury real yields averaged 3.30 percent during the 1990s. The model-based real yield for one-year treasuries is -3.51 percent in March 2022, compared to an average of 2.21 percent during the 1990s (see {\color{green!68!black}\textbf{---}}). 


In [295]:
clean_data = pd.read_csv(data_dir / 'fed_rates_raw.csv', 
                         index_col='date', parse_dates=True)

n = {'RIFLGFCY10_XII_N.B': 'Ten-year',
     'RIFLGFCY05_XII_N.B': 'Five-year'}

df = clean_data[n.keys()].rename(n, axis=1).dropna(subset=['Ten-year'])

data = (df.resample('MS').mean().iloc[:-1].append(df.iloc[-1]))
data.to_csv(data_dir / 'real_rates.csv', index_label='date', 
            float_format='%g')

adj = node_adj(df)
smax = df.iloc[-1].idxmax()
adj[smax] = adj[smax] + 0.35

colors = {'Five-year': 'violet!50!blue!60!white', 'Ten-year': 'green!85!blue'}
date = {series: 'd' if series == smax else None 
        for series in colors.keys()}
nodes  ='\n'.join([end_node(df[series], color, 
                            date=date[series], full_year=True, 
                            digits=2, offset=adj[series]) 
                   for series, color in colors.items()])
write_txt(text_dir / 'real_yield_nodes.txt', nodes)  

ltdt = dtxt(df.index[-1])['day1']
prdt = (dtxt(df.index[-66])['day4'] if df.index[-66].year == df.index[-1].year 
          else dtxt(df.index[-66])['day1'])
ltval = df['Ten-year'].iloc[-1]
ltval5 = df['Five-year'].iloc[-1]
prval = df['Ten-year'].iloc[-66]
prval5 = df['Five-year'].iloc[-66]
cl = c_line(colors['Ten-year'])
cl2 = c_line(colors['Five-year'])
ch10 = value_text(df['Ten-year'].diff(252).iloc[-1], 'increase_by', 
                  ptype='pp', digits=2)
ch5 = value_text(df['Five-year'].diff(252).iloc[-1], 'increase_by', 
                  ptype='pp', digits=2)
text = ('One measure of real interest rates is treasury '+
        'inflation-indexed securities. The yield on these securities '+
        'can be a proxy for the interest rate investors would charge '+
        'for treasuries, without inflation. \n\n'+
        f'On {ltdt}, the real yield on ten year treasuries was {ltval:.2f} '+
        f'percent {cl}, compared to {prval:.2f} percent three months prior, '+
        f'on {prdt}. Five-year treasuries yield {ltval5:.2f} percent in the '+
        f'latest data, and {prval5:.2f} percent three months prior, after '+
        f'adjusting for expected inflation {cl2}. ')
write_txt(text_dir / 'real_rates_basic.txt', text)
print(text)

One measure of real interest rates is treasury inflation-indexed securities. The yield on these securities can be a proxy for the interest rate investors would charge for treasuries, without inflation. 

On March 23, 2022, the real yield on ten year treasuries was -0.62 percent (see {\color{green!85!blue}\textbf{---}}), compared to -0.97 percent three months prior, on December 17, 2021. Five-year treasuries yield -1.23 percent in the latest data, and -1.47 percent three months prior, after adjusting for expected inflation (see {\color{violet!50!blue!60!white}\textbf{---}}). 


/tmp/ipykernel_90747/141321519.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = (df.resample('MS').mean().iloc[:-1].append(df.iloc[-1]))
